In [13]:
import numpy as np
import pandas as pd
import os

In [17]:
DATA_EXTENSION = ".ssv"
ELEMENT_NAMES_EXTENSION = ".elems"


def get_raw_element_names(sample_name: str) -> list[str]:
    with open(f"{sample_name}/{sample_name}{ELEMENT_NAMES_EXTENSION}", "r", encoding="utf-8") as f:
        return f.read().split(",")


def convert_raw_element_names_to_sv12(raw_element_names: list[str]) -> tuple[list[str]]:
    sv1 = [f"{name}_sv1" for name in raw_element_names]
    sv2 = [f"{name}_sv2" for name in raw_element_names]
    return (sv1, sv2,)


def generate_dtype(
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> list[tuple[str]]:
    base_dt_types = [
        ("scheme_state_sv1", "bool"),
        ("scheme_state_sv2", "bool"),
        ("scheme_state", "bool"),
        ("probability", "float64")
    ]
    dt_types_sv1 = [(name, "bool") for name in element_names_sv1]
    dt_types_sv2 = [(name, "bool") for name in element_names_sv2]
    return [*base_dt_types, *dt_types_sv1, *dt_types_sv2]


def read_directory_as_dataframe(sample_name: str, dtype) -> pd.DataFrame:
    return pd.concat(
        map(
            lambda file: pd.DataFrame(np.fromfile(os.path.join(sample_name, file), dtype=dtype)),
            filter(
                lambda file: file.endswith(DATA_EXTENSION),
                os.listdir(sample_name)
            )
        )
    )


def sum_probabilities(sample: pd.DataFrame) -> None:
    p = sample.loc[sample["scheme_state_sv2"] == True, "probability"].sum()
    q = sample.loc[sample["scheme_state_sv2"] == False, "probability"].sum()
    print(f"p = {p}")
    print(f"q = {q}")
    print(f"p + q = {p + q}")


def is_reconfiguration_same_or_better(sample: pd.DataFrame) -> bool:
    inconsistency = sample.loc[(sample["scheme_state_sv1"] == True) & (sample["scheme_state_sv2"] == False)]
    return inconsistency.empty


def sum_fail_probability_per_element(
        sample: pd.DataFrame,
        raw_element_names: list[str],
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> None:
    failed_samples_sv1 = sample.loc[sample["scheme_state_sv1"] == False]
    failed_samples_sv2 = sample.loc[sample["scheme_state_sv2"] == False]
    mask_sv1 = (failed_samples_sv1[element_names_sv1] == False)
    mask_sv2 = (failed_samples_sv2[element_names_sv2] == False)
    probabilities_sv1 = failed_samples_sv1["probability"].values
    probabilities_sv2 = failed_samples_sv2["probability"].values
    fail_probabilities_sv1 = np.sum(mask_sv1.values * probabilities_sv1[:, None], axis=0).tolist()
    fail_probabilities_sv2 = np.sum(mask_sv2.values * probabilities_sv2[:, None], axis=0).tolist()
    for i in range(len(raw_element_names)):
        print(f"{raw_element_names[i]}: sv1 = {fail_probabilities_sv1[i]:.8e}, sv2 = {fail_probabilities_sv2[i]:.8e}")


def sum_fail_count_per_element(
        sample: pd.DataFrame,
        raw_element_names: list[str],
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> None:
    failed_samples_sv1 = sample.loc[sample["scheme_state_sv1"] == False]
    fail_count_sv1 = (failed_samples_sv1[element_names_sv1] == False).sum()
    failed_samples_sv2 = sample.loc[sample["scheme_state_sv2"] == False]
    fail_count_sv2 = (failed_samples_sv2[element_names_sv2] == False).sum()
    for i in range(len(raw_element_names)):
        print(f"{raw_element_names[i]}: sv1 = {fail_count_sv1.iloc[i]}, sv2 = {fail_count_sv2.iloc[i]}")


def get_data(sample_name: str) -> tuple[list[str], list[str], list[str], pd.DataFrame]:
    raw_element_names = get_raw_element_names(sample_name)
    element_names_sv1, element_names_sv2 = convert_raw_element_names_to_sv12(raw_element_names)
    dtype = generate_dtype(element_names_sv1, element_names_sv2)
    samples = read_directory_as_dataframe(sample_name, dtype)
    return (raw_element_names, element_names_sv1, element_names_sv2, dtype, samples)


def process_sample(sample_name: str) -> tuple[list[str], list[str], list[str], pd.DataFrame]:
    raw_element_names, element_names_sv1, element_names_sv2, dtype, samples = get_data(sample_name)

    print("=== sum probabilities ===")
    sum_probabilities(samples)

    if is_reconfiguration_same_or_better(samples):
        print("=== reconfiguration is consistent ===")
    else:
        print("=== reconfiguration is inconsistent ===")
        return

    print("=== sum fail probability(sv1 | sv2) per element ===")
    sum_fail_probability_per_element(
        samples,
        raw_element_names,
        element_names_sv1,
        element_names_sv2
    )

    #print("=== sum fail count(sv1 | sv2) per element ===")
    #sum_fail_count_per_element(
    #    samples,
    #    raw_element_names,
    #    element_names_sv1,
    #    element_names_sv2
    #)

    return (raw_element_names, element_names_sv1, element_names_sv2, samples)

In [3]:
def simple():
    raw_element_names, element_names_sv1, element_names_sv2, samples = process_sample("simple")
#simple()

In [4]:
def process_sample_with_fail_count_per_123(sample_name: str) -> None:
    raw_element_names, element_names_sv1, element_names_sv2, samples = process_sample(sample_name)

    pr_cases_sv1 = (
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False)]
    )

    pr_cases_sv2 = (
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False)]
    )

    print("=== sum fail count(sv1 | sv2) per pr1 * pr2 * pr3 state set ===")
    for i in range(8):
        print(f"case {i}: sv1 = {pr_cases_sv1[i].shape[0]}; sv2 = {pr_cases_sv2[i].shape[0]}")

In [3]:
def process_sample_with_del(sample_name: str) -> None:
    output = process_sample(sample_name)
    del output

In [67]:
def compare_with_manual_no_reconfiguration(sample_name: str) -> None:
    raw_element_names, element_names_sv1, element_names_sv2, _, samples = get_data(sample_name)

    t1 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == False) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == False) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 1 type 1 = {t1.iloc[0]["scheme_state_sv2"]}")

    t2 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == False) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == False) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 2 type 1 = {t2.iloc[0]["scheme_state_sv2"]}")

    t3 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == False) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == False) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 3 type 1 = {t3.iloc[0]["scheme_state_sv2"]}")

    t4 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == False) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == False) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 4 type 1 = {t4.iloc[0]["scheme_state_sv2"]}")

    t5 = samples.loc[
        (samples["d1_sv1"] == False) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == False) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 5 type 1 = {t5.iloc[0]["scheme_state_sv2"]}")

    t6 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == False) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == False) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 6 type 1 = {t6.iloc[0]["scheme_state_sv2"]}")

    t7 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == False) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == False) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 7 type 1 = {t7.iloc[0]["scheme_state_sv2"]}")

    t8 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == False) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == False)
    ]
    print(f"vector 8 type 1 = {t8.iloc[0]["scheme_state_sv2"]}")

    t9 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == False) & (samples["c2_sv1"] == False) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == False) & (samples["c2_sv2"] == False) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 1 type 2 = {t9.iloc[0]["scheme_state_sv2"]}")

    t10 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == False) & (samples["m2_sv1"] == False) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == False) & (samples["m2_sv2"] == False)
    ]
    print(f"vector 2 type 2 = {t10.iloc[0]["scheme_state_sv2"]}")

    t11 = samples.loc[
        (samples["d1_sv1"] == False) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == False) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == False) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == False) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 3 type 2 = {t11.iloc[0]["scheme_state_sv2"]}")

    # wtf!!!
    t12 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == False) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == False) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == False) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == False) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 4 type 2 = {t12.iloc[0]["scheme_state_sv2"]}")

    t13 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == False) & (samples["d3_sv1"] == False) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == False) & (samples["d3_sv2"] == False) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 5 type 2 = {t13.iloc[0]["scheme_state_sv2"]}")

    t14 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == False) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == False) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == False) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == False) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 5 type 2 = {t14.iloc[0]["scheme_state_sv2"]}")

    t15 = samples.loc[
        (samples["d1_sv1"] == False) & (samples["d2_sv1"] == False) & (samples["d3_sv1"] == False) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == False) & (samples["d2_sv2"] == False) & (samples["d3_sv2"] == False) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 1 type 3 = {t15.iloc[0]["scheme_state_sv2"]}")

    t16 = samples.loc[
        (samples["d1_sv1"] == False) & (samples["d2_sv1"] == False) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == False) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == False) & (samples["d2_sv2"] == False) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == False) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 2 type 3 = {t16.iloc[0]["scheme_state_sv2"]}")

    t17 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == False) & (samples["d3_sv1"] == False) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == False) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == False) & (samples["d3_sv2"] == False) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == False) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 3 type 3 = {t17.iloc[0]["scheme_state_sv2"]}")

    t18 = samples.loc[
        (samples["d1_sv1"] == False) & (samples["d2_sv1"] == False) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == False) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == False) & (samples["d2_sv2"] == False) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == False) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 4 type 3 = {t18.iloc[0]["scheme_state_sv2"]}")

    t19 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == False) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == False) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == False) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == False) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == False) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == False) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 5 type 3 = {t19.iloc[0]["scheme_state_sv2"]}")

    t20 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == False) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == False) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == False) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == False) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == False) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == False)
    ]
    print(f"vector 6 type 3 = {t20.iloc[0]["scheme_state_sv2"]}")


print("v2 compare (no reconfiguration needed)")

#print("=== greedy ===")
#compare_with_manual_no_reconfiguration("s23-77788-greedy")

print("=== brute ===")
compare_with_manual_no_reconfiguration("s23-77788-brute")

v2 compare (no reconfiguration needed)
=== brute ===
vector 1 type 1 = False
vector 2 type 1 = False
vector 3 type 1 = False
vector 4 type 1 = False
vector 5 type 1 = True
vector 6 type 1 = True
vector 7 type 1 = True
vector 8 type 1 = True
vector 1 type 2 = False
vector 2 type 2 = False
vector 3 type 2 = False
vector 4 type 2 = False
vector 5 type 2 = True
vector 5 type 2 = True
vector 1 type 3 = False
vector 2 type 3 = False
vector 3 type 3 = False
vector 4 type 3 = True
vector 5 type 3 = True
vector 6 type 3 = True


In [60]:
def compare_with_manual_original_reconfiguration(sample_name: str) -> None:
    raw_element_names, element_names_sv1, element_names_sv2, _, samples = get_data(sample_name)

    t1 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 1 = {t1.iloc[0]["scheme_state_sv2"]}")

    t2 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == False) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == False) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 2 = {t2.iloc[0]["scheme_state_sv2"]}")


print("v2 compare (original reconfiguration)")

#print("=== greedy ===")
#compare_with_manual_original_reconfiguration("s23-original-greedy")

print("=== brute ===")
compare_with_manual_original_reconfiguration("s23-original-brute")


v2 compare (original reconfiguration)
=== brute ===
vector 1 = True
vector 2 = False


In [66]:
def compare_with_manual_advanced_reconfiguration(sample_name: str) -> None:
    raw_element_names, element_names_sv1, element_names_sv2, _, samples = get_data(sample_name)

    t1 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 1 type 1 = {t1.iloc[0]["scheme_state_sv2"]}")

    t2 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == False) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 2 type 1 = {t2.iloc[0]["scheme_state_sv2"]}")

    t3 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 1 type 2 = {t3.iloc[0]["scheme_state_sv2"]}")

    t4 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == False) & (samples["pr5_sv1"] == False) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 2 type 2 = {t4.iloc[0]["scheme_state_sv2"]}")

    t5 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == False) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 3 type 2 = {t5.iloc[0]["scheme_state_sv2"]}")

    t6 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 1 type 3 = {t6.iloc[0]["scheme_state_sv2"]}")

    t7 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == True) & (samples["pr5_sv1"] == False) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 2 type 3 = {t7.iloc[0]["scheme_state_sv2"]}")

    t8 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == False) & (samples["pr5_sv1"] == False) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == False) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 3 type 3 = {t8.iloc[0]["scheme_state_sv2"]}")

    t9 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False) & (samples["pr4_sv1"] == False) & (samples["pr5_sv1"] == True) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False) & (samples["pr4_sv2"] == True) & (samples["pr5_sv2"] == True) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 1 type 4 = {t9.iloc[0]["scheme_state_sv2"]}")

    t10 = samples.loc[
        (samples["d1_sv1"] == True) & (samples["d2_sv1"] == True) & (samples["d3_sv1"] == True) & (samples["d6_sv1"] == True) & (samples["d8_sv1"] == True) & (samples["c1_sv1"] == True) & (samples["c2_sv1"] == True) & (samples["c4_sv1"] == True) & (samples["c5_sv1"] == True) & (samples["c6_sv1"] == True) & (samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True) & (samples["pr4_sv1"] == False) & (samples["pr5_sv1"] == False) & (samples["a1_sv1"] == True) & (samples["a2_sv1"] == True) & (samples["b1_sv1"] == True) & (samples["b2_sv1"] == True) & (samples["b4_sv1"] == True) & (samples["b5_sv1"] == True) & (samples["m1_sv1"] == True) & (samples["m2_sv1"] == True) &
        (samples["d1_sv2"] == True) & (samples["d2_sv2"] == True) & (samples["d3_sv2"] == True) & (samples["d6_sv2"] == True) & (samples["d8_sv2"] == True) & (samples["c1_sv2"] == True) & (samples["c2_sv2"] == True) & (samples["c4_sv2"] == True) & (samples["c5_sv2"] == True) & (samples["c6_sv2"] == True) & (samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True) & (samples["pr4_sv2"] == False) & (samples["pr5_sv2"] == False) & (samples["a1_sv2"] == True) & (samples["a2_sv2"] == True) & (samples["b1_sv2"] == True) & (samples["b2_sv2"] == True) & (samples["b4_sv2"] == True) & (samples["b5_sv2"] == True) & (samples["m1_sv2"] == True) & (samples["m2_sv2"] == True)
    ]
    print(f"vector 2 type 4 = {t10.iloc[0]["scheme_state_sv2"]}")


print("v2 compare (advanced reconfiguration)")

print("=== brute ===")
compare_with_manual_advanced_reconfiguration("s23-77788-brute")


v2 compare (advanced reconfiguration)
=== brute ===
vector 1 type 1 = True
vector 2 type 1 = True
vector 1 type 2 = True
vector 2 type 2 = True
vector 3 type 2 = True
vector 1 type 3 = False
vector 2 type 3 = False
vector 3 type 3 = False
vector 1 type 4 = False
vector 2 type 4 = False


In [6]:
process_sample_with_fail_count_per_123("s23-original-greedy")

=== sum probabilities ===
p = 0.9990208569798807
q = 0.0009791430201191219
p + q = 0.9999999999999999
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 2.87982720e-08
pr2: sv1 = 1.20000000e-04, sv2 = 2.87982720e-08
pr3: sv1 = 1.20000000e-04, sv2 = 2.87982720e-08
pr4: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
pr5: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 2.00805635e-08, sv2 = 1.46871453e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 7.16675097e-07, sv2 = 5.69315417e-07
c2: sv1 = 7.16675097e-07, sv2 = 5.69315417e-07
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 7.16674899e-07, sv2 = 5.69315219e-07
c6: sv1 = 7.16674899e-07, sv2 = 5.69315219e-07
d1: sv1 = 2.94517019e-08, sv2 = 2.15413552e-08
d2: 

In [7]:
process_sample_with_fail_count_per_123("s23-original-brute")

=== sum probabilities ===
p = 0.9990208569798807
q = 0.0009791430201191219
p + q = 0.9999999999999999
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 2.87982720e-08
pr2: sv1 = 1.20000000e-04, sv2 = 2.87982720e-08
pr3: sv1 = 1.20000000e-04, sv2 = 2.87982720e-08
pr4: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
pr5: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 2.00805635e-08, sv2 = 1.46871453e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 7.16675097e-07, sv2 = 5.69315417e-07
c2: sv1 = 7.16675097e-07, sv2 = 5.69315417e-07
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 7.16674899e-07, sv2 = 5.69315219e-07
c6: sv1 = 7.16674899e-07, sv2 = 5.69315219e-07
d1: sv1 = 2.94517019e-08, sv2 = 2.15413552e-08
d2: 

In [8]:
process_sample_with_fail_count_per_123("s23-77788-greedy")

=== sum probabilities ===
p = 0.9992607082975082
q = 0.0007392917024916249
p + q = 0.9999999999999999
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr2: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr3: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr4: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
pr5: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 2.00805635e-08, sv2 = 1.10893755e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 7.16675097e-07, sv2 = 4.71016679e-07
c2: sv1 = 7.16675097e-07, sv2 = 4.71016679e-07
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 7.16674899e-07, sv2 = 4.71016481e-07
c6: sv1 = 7.16674899e-07, sv2 = 4.71016481e-07
d1: sv1 = 2.94517019e-08, sv2 = 1.62646263e-08
d2: 

In [9]:
process_sample_with_fail_count_per_123("s23-77788-brute")

=== sum probabilities ===
p = 0.9992607082975082
q = 0.0007392917024916249
p + q = 0.9999999999999999
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.72800000e-12
pr2: sv1 = 1.20000000e-04, sv2 = 8.63875589e-12
pr3: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
pr4: sv1 = 1.20000000e-04, sv2 = 6.22030234e-16
pr5: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 2.00805635e-08, sv2 = 1.10893755e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 7.16675097e-07, sv2 = 4.71016679e-07
c2: sv1 = 7.16675097e-07, sv2 = 4.71016679e-07
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 7.16674899e-07, sv2 = 4.71016481e-07
c6: sv1 = 7.16674899e-07, sv2 = 4.71016481e-07
d1: sv1 = 2.94517019e-08, sv2 = 1.62646263e-08
d2: 

In [10]:
process_sample_with_del("s23-77788-modified-connections-greedy")

=== sum probabilities ===
p = 0.999737721521639
q = 0.0002622784783607359
p + q = 0.9999999999999997
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr2: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr3: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr4: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
pr5: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.31544599e-08, sv2 = 4.15911479e-09
b2: sv1 = 1.31544599e-08, sv2 = 4.15911479e-09
b4: sv1 = 1.31544599e-08, sv2 = 4.15911479e-09
b5: sv1 = 1.31544599e-08, sv2 = 4.15911479e-09
c1: sv1 = 3.53479464e-07, sv2 = 1.07603051e-07
c2: sv1 = 3.53479464e-07, sv2 = 1.07603051e-07
c4: sv1 = 3.53479464e-07, sv2 = 1.07603051e-07
c5: sv1 = 5.21296899e-07, sv2 = 2.75521212e-07
c6: sv1 = 5.21296899e-07, sv2 = 2.75521212e-07
d1: sv1 = 1.89634973e-08, sv2 = 5.77012652e-09
d2: s

In [11]:
process_sample_with_del("s23-77788-modified-connections-brute")

=== sum probabilities ===
p = 0.999737721521639
q = 0.0002622784783607359
p + q = 0.9999999999999997
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.72800000e-12
pr2: sv1 = 1.20000000e-04, sv2 = 8.63875589e-12
pr3: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
pr4: sv1 = 1.20000000e-04, sv2 = 6.22030234e-16
pr5: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.31544599e-08, sv2 = 4.15911479e-09
b2: sv1 = 1.31544599e-08, sv2 = 4.15911479e-09
b4: sv1 = 1.31544599e-08, sv2 = 4.15911479e-09
b5: sv1 = 1.31544599e-08, sv2 = 4.15911479e-09
c1: sv1 = 3.53479464e-07, sv2 = 1.07603051e-07
c2: sv1 = 3.53479464e-07, sv2 = 1.07603051e-07
c4: sv1 = 3.53479464e-07, sv2 = 1.07603051e-07
c5: sv1 = 5.21296899e-07, sv2 = 2.75521212e-07
c6: sv1 = 5.21296899e-07, sv2 = 2.75521212e-07
d1: sv1 = 1.89634973e-08, sv2 = 5.77012652e-09
d2: s

In [12]:
process_sample_with_del("s24-d9-right-greedy")

=== sum probabilities ===
p = 0.9997597157515122
q = 0.0002402842484872624
p + q = 0.9999999999999994
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr2: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr3: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr4: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
pr5: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.28247493e-08, sv2 = 3.82920629e-09
b2: sv1 = 1.28247493e-08, sv2 = 3.82920629e-09
b4: sv1 = 1.28247493e-08, sv2 = 3.82920629e-09
b5: sv1 = 1.28247493e-08, sv2 = 3.82920629e-09
c1: sv1 = 3.44467241e-07, sv2 = 9.85854181e-08
c2: sv1 = 3.44467241e-07, sv2 = 9.85854181e-08
c4: sv1 = 3.44467241e-07, sv2 = 9.85854181e-08
c5: sv1 = 5.12288368e-07, sv2 = 2.66507274e-07
c6: sv1 = 5.12288368e-07, sv2 = 2.66507274e-07
d1: sv1 = 1.84799145e-08, sv2 = 5.28625347e-09
d2: 

In [13]:
process_sample_with_del("s24-d9-right-brute")

=== sum probabilities ===
p = 0.9997597157515122
q = 0.0002402842484872624
p + q = 0.9999999999999994
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.72800000e-12
pr2: sv1 = 1.20000000e-04, sv2 = 8.63875589e-12
pr3: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
pr4: sv1 = 1.20000000e-04, sv2 = 6.22030234e-16
pr5: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.28247493e-08, sv2 = 3.82920629e-09
b2: sv1 = 1.28247493e-08, sv2 = 3.82920629e-09
b4: sv1 = 1.28247493e-08, sv2 = 3.82920629e-09
b5: sv1 = 1.28247493e-08, sv2 = 3.82920629e-09
c1: sv1 = 3.44467241e-07, sv2 = 9.85854181e-08
c2: sv1 = 3.44467241e-07, sv2 = 9.85854181e-08
c4: sv1 = 3.44467241e-07, sv2 = 9.85854181e-08
c5: sv1 = 5.12288368e-07, sv2 = 2.66507274e-07
c6: sv1 = 5.12288368e-07, sv2 = 2.66507274e-07
d1: sv1 = 1.84799145e-08, sv2 = 5.28625347e-09
d2: 

In [14]:
process_sample_with_del("s25-d9-d10-right-greedy")

=== sum probabilities ===
p = 0.999759716235385
q = 0.00024028376461420454
p + q = 0.9999999999999991
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr2: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr3: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr4: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
pr5: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.28247421e-08, sv2 = 3.82919903e-09
b2: sv1 = 1.28247421e-08, sv2 = 3.82919903e-09
b4: sv1 = 1.28247421e-08, sv2 = 3.82919903e-09
b5: sv1 = 1.28247421e-08, sv2 = 3.82919903e-09
c1: sv1 = 3.44467043e-07, sv2 = 9.85852197e-08
c2: sv1 = 3.44467043e-07, sv2 = 9.85852197e-08
c4: sv1 = 3.44467043e-07, sv2 = 9.85852197e-08
c5: sv1 = 5.12288170e-07, sv2 = 2.66507076e-07
c6: sv1 = 5.12288170e-07, sv2 = 2.66507076e-07
d1: sv1 = 1.84799038e-08, sv2 = 5.28624282e-09
d2: 

In [15]:
process_sample_with_del("s25-d9-d10-right-brute")

=== sum probabilities ===
p = 0.999759716235385
q = 0.00024028376461420454
p + q = 0.9999999999999991
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.72800000e-12
pr2: sv1 = 1.20000000e-04, sv2 = 8.63875589e-12
pr3: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
pr4: sv1 = 1.20000000e-04, sv2 = 6.22030234e-16
pr5: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.28247421e-08, sv2 = 3.82919903e-09
b2: sv1 = 1.28247421e-08, sv2 = 3.82919903e-09
b4: sv1 = 1.28247421e-08, sv2 = 3.82919903e-09
b5: sv1 = 1.28247421e-08, sv2 = 3.82919903e-09
c1: sv1 = 3.44467043e-07, sv2 = 9.85852197e-08
c2: sv1 = 3.44467043e-07, sv2 = 9.85852197e-08
c4: sv1 = 3.44467043e-07, sv2 = 9.85852197e-08
c5: sv1 = 5.12288170e-07, sv2 = 2.66507076e-07
c6: sv1 = 5.12288170e-07, sv2 = 2.66507076e-07
d1: sv1 = 1.84799038e-08, sv2 = 5.28624282e-09
d2: 

In [16]:
process_sample_with_del("s27-d9-d10-c7-right-c8-left-greedy")

=== sum probabilities ===
p = 0.9997598842949936
q = 0.00024011570500590327
p + q = 0.9999999999999996
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr2: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr3: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr4: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
pr5: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.28222227e-08, sv2 = 3.82667817e-09
b2: sv1 = 1.28222227e-08, sv2 = 3.82667817e-09
b4: sv1 = 1.28222227e-08, sv2 = 3.82667817e-09
b5: sv1 = 1.28222227e-08, sv2 = 3.82667817e-09
c1: sv1 = 3.44329373e-07, sv2 = 9.84474673e-08
c2: sv1 = 3.44329373e-07, sv2 = 9.84474673e-08
c4: sv1 = 3.44329373e-07, sv2 = 9.84474673e-08
c5: sv1 = 3.44398180e-07, sv2 = 9.85163153e-08
c6: sv1 = 3.44398180e-07, sv2 = 9.85163153e-08
d1: sv1 = 1.84762087e-08, sv2 = 5.28254551e-09
d2:

In [17]:
process_sample_with_del("s27-d9-d10-c7-right-c8-left-brute")

=== sum probabilities ===
p = 0.9997598842949936
q = 0.00024011570500590327
p + q = 0.9999999999999996
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.72800000e-12
pr2: sv1 = 1.20000000e-04, sv2 = 8.63875589e-12
pr3: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
pr4: sv1 = 1.20000000e-04, sv2 = 6.22030234e-16
pr5: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.28222227e-08, sv2 = 3.82667817e-09
b2: sv1 = 1.28222227e-08, sv2 = 3.82667817e-09
b4: sv1 = 1.28222227e-08, sv2 = 3.82667817e-09
b5: sv1 = 1.28222227e-08, sv2 = 3.82667817e-09
c1: sv1 = 3.44329373e-07, sv2 = 9.84474673e-08
c2: sv1 = 3.44329373e-07, sv2 = 9.84474673e-08
c4: sv1 = 3.44329373e-07, sv2 = 9.84474673e-08
c5: sv1 = 3.44398180e-07, sv2 = 9.85163153e-08
c6: sv1 = 3.44398180e-07, sv2 = 9.85163153e-08
d1: sv1 = 1.84762087e-08, sv2 = 5.28254551e-09
d2:

In [18]:
process_sample_with_del("s29-d9-d10-c7-right-c8-left-a4-greedy")

=== sum probabilities ===
p = 0.9999998410637558
q = 1.5893623277071066e-07
p + q = 0.9999999999999886
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr2: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr3: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr4: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
pr5: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
a1: sv1 = 8.63914139e-08, sv2 = 1.44173423e-08
a2: sv1 = 8.63914139e-08, sv2 = 1.44173423e-08
b1: sv1 = 9.22508424e-09, sv2 = 2.27380633e-10
b2: sv1 = 9.22508424e-09, sv2 = 2.27380633e-10
b4: sv1 = 9.22508424e-09, sv2 = 2.27380633e-10
b5: sv1 = 9.22508424e-09, sv2 = 2.27380633e-10
c1: sv1 = 2.46006113e-07, sv2 = 6.51921015e-11
c2: sv1 = 2.46006113e-07, sv2 = 6.51921015e-11
c4: sv1 = 2.46006113e-07, sv2 = 6.51921015e-11
c5: sv1 = 2.46074936e-07, sv2 = 1.34056587e-10
c6: sv1 = 2.46074936e-07, sv2 = 1.34056587e-10
d1: sv1 = 1.32003265e-08, sv2 = 3.49659736e-12
d2:

In [19]:
process_sample_with_del("s29-d9-d10-c7-right-c8-left-a4-brute")

=== sum probabilities ===
p = 0.9999998410637558
q = 1.5893623277071066e-07
p + q = 0.9999999999999886
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.72800000e-12
pr2: sv1 = 1.20000000e-04, sv2 = 8.63875589e-12
pr3: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
pr4: sv1 = 1.20000000e-04, sv2 = 6.22030234e-16
pr5: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
a1: sv1 = 8.63914139e-08, sv2 = 1.44173423e-08
a2: sv1 = 8.63914139e-08, sv2 = 1.44173423e-08
b1: sv1 = 9.22508424e-09, sv2 = 2.27380633e-10
b2: sv1 = 9.22508424e-09, sv2 = 2.27380633e-10
b4: sv1 = 9.22508424e-09, sv2 = 2.27380633e-10
b5: sv1 = 9.22508424e-09, sv2 = 2.27380633e-10
c1: sv1 = 2.46006113e-07, sv2 = 6.51921015e-11
c2: sv1 = 2.46006113e-07, sv2 = 6.51921015e-11
c4: sv1 = 2.46006113e-07, sv2 = 6.51921015e-11
c5: sv1 = 2.46074936e-07, sv2 = 1.34056587e-10
c6: sv1 = 2.46074936e-07, sv2 = 1.34056587e-10
d1: sv1 = 1.32003265e-08, sv2 = 3.49659736e-12
d2:

In [5]:
process_sample_with_del("s26-final-greedy")

=== sum probabilities ===
p = 0.9999996724798291
q = 3.2752016698965956e-07
p + q = 0.9999999999999961
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr2: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr3: sv1 = 1.20000000e-04, sv2 = 1.38217191e-11
pr4: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
pr5: sv1 = 1.20000000e-04, sv2 = 1.55488898e-11
a1: sv1 = 8.64116295e-08, sv2 = 1.44375699e-08
a2: sv1 = 8.64116295e-08, sv2 = 1.44375699e-08
b1: sv1 = 9.22761144e-09, sv2 = 2.29909354e-10
b2: sv1 = 9.22761144e-09, sv2 = 2.29909354e-10
b4: sv1 = 9.22761144e-09, sv2 = 2.29909354e-10
b5: sv1 = 9.22761144e-09, sv2 = 2.29909354e-10
c1: sv1 = 2.46144014e-07, sv2 = 2.03175988e-10
c2: sv1 = 2.46144014e-07, sv2 = 2.03175988e-10
c4: sv1 = 2.46144014e-07, sv2 = 2.03175988e-10
c5: sv1 = 4.14005421e-07, sv2 = 1.68165335e-07
c6: sv1 = 4.14005421e-07, sv2 = 1.68165335e-07
d1: sv1 = 1.32040331e-08, sv2 = 7.20544391e-12
d2:

In [6]:
process_sample_with_del("s26-final-brute")

=== sum probabilities ===
p = 0.9999996724798291
q = 3.2752016698965956e-07
p + q = 0.9999999999999961
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.20000000e-04, sv2 = 1.72800000e-12
pr2: sv1 = 1.20000000e-04, sv2 = 8.63875589e-12
pr3: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
pr4: sv1 = 1.20000000e-04, sv2 = 6.22030234e-16
pr5: sv1 = 1.20000000e-04, sv2 = 5.18337794e-12
a1: sv1 = 8.64116295e-08, sv2 = 1.44375699e-08
a2: sv1 = 8.64116295e-08, sv2 = 1.44375699e-08
b1: sv1 = 9.22761144e-09, sv2 = 2.29909354e-10
b2: sv1 = 9.22761144e-09, sv2 = 2.29909354e-10
b4: sv1 = 9.22761144e-09, sv2 = 2.29909354e-10
b5: sv1 = 9.22761144e-09, sv2 = 2.29909354e-10
c1: sv1 = 2.46144014e-07, sv2 = 2.03175988e-10
c2: sv1 = 2.46144014e-07, sv2 = 2.03175988e-10
c4: sv1 = 2.46144014e-07, sv2 = 2.03175988e-10
c5: sv1 = 4.14005421e-07, sv2 = 1.68165335e-07
c6: sv1 = 4.14005421e-07, sv2 = 1.68165335e-07
d1: sv1 = 1.32040331e-08, sv2 = 7.20544391e-12
d2: